# Homework 6 Question 3: Heat pipe design.
A heated fluid at temperature $T$ (degrees above ambient temperature)
flows in a pipe with fixed length and circular cross section with radius $r$. A layer of insulation, with
thickness $w$, surrounds the pipe to reduce heat loss through the pipe walls ($w$ is much smaller than $r$).
The design variables in this problem are $T, r$, and $w$.

The energy cost due to heat loss is roughly equal to $α_1T r/w$. The cost of the pipe, which has a fixed
wall thickness, is approximately proportional to the total material, i.e., it is given by $α_2r$. The cost of
the insulation is also approximately proportional to the total insulation material, i.e., roughly $α_3rw$.
The total cost is the sum of these three costs.

The heat flow down the pipe is entirely due to the flow of the fluid, which has a fixed velocity, i.e., it
is given by $α_4T r^2$
. The constants $α_i$ are all positive, as are the variables $T, r,$ and $w$.

Now the problem: maximize the total heat flow down the pipe, subject to an upper limit C_max on total
cost, and the constraints 

$$
    T_{min} ≤ T ≤ T_{max}, \space r_{min} ≤ r ≤ r_{max}, \space w_{min} ≤ w ≤ w_{max}, \space w ≤ 0.1r
$$

__a) Express this problem as a geometric program, and convert it into a convex optimization problem.__

Equivalent G.P.:
\begin{equation*}
\begin{aligned}
&{\text{minimize}}
& & (1/α_4)T^{−1}r^{−2} \\
& \text{subject to}:
& & (1/T_{max})T ≤ 1, T_{min}T^{−1} ≤ 1  \\
& & & (1/r_{max})r ≤ 1, r_{min}r^{−1} ≤ 1  \\
& & & (1/w_{max})w ≤ 1, w_{min}w^{−1} ≤ 1  \\
& & & 10wr^{−1} ≤ 1.
\end{aligned}
\end{equation*}







Let  $x = log(T), y = log(r) , z = log(w)$

So objective function

$$α_4Tr^2 : log(e^{log(\alpha_4)+x+2y})$$

Constraints

$$T_{min} ≤ T ≤ T_{max} : \space log(e^{log(T_{min})-x}) \leq 0 ,\space log(e^{x+log(1/T_{max})}) \leq 0$$
 $$r_{min} ≤ r ≤ r_{max} : \space log(e^{log(r_{min})-y}) \leq 0, \space log(e^{y+log(1/r_{max})}) \leq 0$$
$$w_{min} ≤ w ≤ w_{max} : \space log(e^{log(w_{min})-z}) \leq 0, \space log(e^{z+log(1/w_{max})}) \leq 0$$
$$ w ≤ 0.1r : log (e^{log(10) + z - y}) \leq 0$$

__Heat Loss Cost__ $(α_1Trw^{-1}) => log(e^{log(\alpha_1) + x + y - z})$

__Pipe Cost__ $(α_2r) => log(e^{log(\alpha_2) + y})$

__Insulation Cost__ $(α_3rw) => log(e^{log(\alpha_3) + x + z})$

__b) Consider a simple instance of this problem, where $Cmax = 500$ and $α_1 = α_2 = α_3 = α_4 = 1$.
Also assume for simplicity that each variable has a lower bound of zero and no upper bound.
Solve this problem using JuMP. Use the Mosek solver and the command @NLconstraint(...)
to specify nonlinear constraints such as log-sum-exp functions. Note: Mosek can solve general
convex optimization problems! What is the optimal $T, r$, and $w$?__

In [7]:
using JuMP, Mosek

α1 = 1
α2 = 1
α3 = 1
α4 = 1
Cmax = 500

m = Model(solver=MosekSolver(LOG=0))

@variable(m, x >= 0)
@variable(m, y >= 0)
@variable(m, z >= 0)

@NLexpression(m, heatLossCost, log(α1)+x+y-z)
@NLexpression(m, pipeCost, log(α2) + y)
@NLexpression(m, insulationCost, log(α3) + x + z)

@NLconstraint(m, log(10) + z - y  <= 0)
@NLconstraint(m, heatLossCost + pipeCost + insulationCost <= Cmax)

@NLexpression(m, heatFlow, log(α4) + x + 2y)

@NLobjective(m, Max, heatFlow)
println("Status:", solve(m))
X = getvalue(x)
Y = getvalue(y)
Z = getvalue(z)

T = e^X
r = e^Y
w = e^Z
println("Optimal Temperature: ", T, ", Radius: ",r, ", Width: ",w)
println("Max heat flow: ", α4*T*r^2)

Status:Optimal
Optimal Temperature: 1.0000001730422694, Radius: 3.7464406250722294e108, Width: 6.120817586443607e53
Max heat flow: 1.4035819785981284e217
